In [1]:
# Directories
import os

new_directory = r'c://Users//Fer//TESIS_ARCHIVOS//TESIS_AIRE//MP_Forecasting//aqi_forecasting//notebooks'
os.chdir(new_directory)

# Data Manipulation
import pandas as pd # for data manipulation
import numpy as np # for data manipulation


# Training utils
from training_code.utils import utils_xgboost

# Optuna
import optuna
import joblib
import pickle

# Tiempo
import datetime as dt
from dateutil.relativedelta import relativedelta, MO

# Modelos
from sklearn.linear_model import LinearRegression # for building a linear regression model
from sklearn.svm import SVR # for building SVR model
from sklearn.preprocessing import MinMaxScaler
from sklearn.multioutput import MultiOutputRegressor
import xgboost as xgb
from sklearn.model_selection import train_test_split

# Metricas
from sklearn.metrics import mean_absolute_error #MAE
from sklearn.metrics import mean_absolute_percentage_error #MAPE
from sklearn.metrics import mean_squared_error #MSE, para RMSE: squared = False

# Visualizations
import plotly.graph_objects as go # for data visualization
import plotly.express as px # for data visualization
import matplotlib.pyplot as plt

# Advertencias
import warnings
warnings.filterwarnings("ignore")

c:\Users\Fer\anaconda3\lib\site-packages\xgboost\compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [4]:
train_ee = pd.read_csv('datos/230127_train_ESTACIONES.csv', parse_dates=['FECHAHORA'])
test_ee = pd.read_csv('datos/230127_test_ESTACIONES.csv', parse_dates=['FECHAHORA'])

In [ ]:
test_ee.info()

In [21]:
junto = [train_ee, test_ee]

datos = pd.concat(junto)

datos.head()

,ESTACION,FECHAHORA,ANHO,DIA,MES,HORA,MINUTO,MP1,MP2_5,MP10,...,DIA_SEM,AQI_MP2_5,AQI_MP10,MEDICION_DIA,MP1_ANTERIOR,MP2_5_ANTERIOR,MP10_ANTERIOR,TEMPERATURA_PRONOSTICO,HUMEDAD_PRONOSTICO,PRESION_PRONOSTICO
0,1,2019-12-30 13:00:00,2019,30,12,13,0,2.839,3.989,4.599,...,0,72.0,25.0,157,2.609,3.720,4.519,36.5,42.8,995.1
1,1,2019-12-30 13:05:00,2019,30,12,13,5,9.000,13.079,21.319,...,0,72.0,25.0,158,1.919,2.659,10.340,36.7,42.1,994.9
2,1,2019-12-30 13:10:00,2019,30,12,13,10,1.969,2.369,13.170,...,0,72.0,25.0,159,1.769,2.609,3.419,36.6,43.5,994.8
3,1,2019-12-30 13:15:00,2019,30,12,13,15,1.740,2.379,2.429,...,0,72.0,25.0,160,1.480,2.009,2.159,37.0,41.4,994.7
4,1,2019-12-30 13:20:00,2019,30,12,13,20,2.710,4.119,7.710,...,0,72.0,25.0,161,1.250,1.909,2.809,37.1,40.6,994.6


In [22]:
e4 = datos[datos['ESTACION'] == 4]

e4.FECHAHORA.max()

e4.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 158113 entries, 394563 to 105983
Data columns (total 28 columns):
 #   Column                  Non-Null Count   Dtype         
---  ------                  --------------   -----         
 0   ESTACION                158113 non-null  int64         
 1   FECHAHORA               158113 non-null  datetime64[ns]
 2   ANHO                    158113 non-null  int64         
 3   DIA                     158113 non-null  int64         
 4   MES                     158113 non-null  int64         
 5   HORA                    158113 non-null  int64         
 6   MINUTO                  158113 non-null  int64         
 7   MP1                     158113 non-null  float64       
 8   MP2_5                   158113 non-null  float64       
 9   MP10                    158113 non-null  float64       
 10  TEMPERATURA             158113 non-null  float64       
 11  HUMEDAD                 158113 non-null  float64       
 12  PRESION                 1

# 6 horas - training

In [41]:
estacion = 4

mape = []
rmse = []
mae = []

mape_t = []
rmse_t = []
mae_t = []

variables = ['MP1', 'MP2_5', 'MP10', 'AQI_MP10', 'AQI_MP2_5']
dependent = ['AQI_MP2_5']

number_of_features = len(variables)

forecast_days = 0.25
training_days = 2
samples_per_day = 288

step = forecast_days*samples_per_day

input_samples = int(samples_per_day * training_days) # cantidad de muestras en 7 dias
output_samples = int(samples_per_day * forecast_days) # cantidad de muestras en 1 dia
train_test_samples = int(input_samples + output_samples) # cantidad de datos para el train_test




train_set_1 = datos[(datos['FECHAHORA']) < (datos.FECHAHORA.min() + relativedelta(months=3))]
# print(train_set_1.info())
# print(train_set_1.FECHAHORA.min())

train_set_2 = datos[datos['FECHAHORA'] < (datos.FECHAHORA.min() + relativedelta(months=6))]
# print(train_set_2.info())
# print(train_set_2.FECHAHORA.min())

train_set_3 = datos[datos['FECHAHORA'] < (datos.FECHAHORA.min() + relativedelta(months=9))]
# print(train_set_3.FECHAHORA.min())

train_set_4 = datos[datos['FECHAHORA'] < (datos.FECHAHORA.min() + relativedelta(months=12))]
#print(train_set_4.FECHAHORA.min())

train_set_5 = datos[datos['FECHAHORA'] < (datos.FECHAHORA.min() + relativedelta(years = 1, months = 3))]
#print(train_set_5.FECHAHORA.min())

test_set_1 = datos[(datos['FECHAHORA'] < (datos.FECHAHORA.min() + relativedelta(months=6)))]

test_set_2 = datos[(datos['FECHAHORA'] < (datos.FECHAHORA.min() + relativedelta(months=9)))]

test_set_3 = datos[(datos['FECHAHORA'] < (datos.FECHAHORA.min() + relativedelta(months=12)))]

test_set_4 = datos[(datos['FECHAHORA'] < (datos.FECHAHORA.min() + relativedelta(months=15)))]

test_set_5 = datos

set_set = [[train_set_1, 2, test_set_1, 3], [train_set_2, 5, test_set_2, 6], [train_set_3, 8, test_set_3, 9], 
           [train_set_4, 11, test_set_4, 12], [train_set_5, 14, test_set_5, 15]]

for i in range(0,5):

    train_months = relativedelta(months = set_set[i][1])
    df_train = set_set[i][0]

    X_train, y_train, X_test, y_test = utils_xgboost.get_everything(df_train, 
                                                        estacion,
                                                        train_months, 
                                                        variables, 
                                                        dependent, 
                                                        train_test_samples, 
                                                        input_samples, 
                                                        output_samples, 
                                                        number_of_features,
                                                        step)
    
    params = {'max_depth': 5,
              'learning_rate': 0.024093830590526,
              'n_estimators': 224,
              'min_child_weight': 6,
              'gamma':  0.005342041600216,
              'subsample': 0.635588411140476,
              'colsample_bytree': 0.998857049787802}
    
    xgb_model = xgb.XGBRegressor(** params)

    trained_xgb_model = MultiOutputRegressor(xgb_model).fit(X_train , y_train)

    # validation

    prediction = trained_xgb_model.predict(X_test)

    mape_val = mean_absolute_percentage_error(prediction, y_test)
    mae_val = mean_absolute_error(prediction, y_test)
    rmse_val = mean_squared_error(prediction, y_test, squared = False)

    print('val MAPE, MAE, RMSE:')

    print(mape_val)
    print(mae_val)
    print(rmse_val)

    mape.append(mape_val)
    mae.append(mae_val)
    rmse.append(rmse_val)

    # testing
    
    train_months = relativedelta(months = set_set[i][3])
    df_train = set_set[i][2]

    X_train, y_train, X_test, y_test = utils_xgboost.get_everything(df_train, 
                                                        estacion,
                                                        train_months, 
                                                        variables, 
                                                        dependent, 
                                                        train_test_samples, 
                                                        input_samples, 
                                                        output_samples, 
                                                        number_of_features,
                                                        step)
    
    # # params = {'max_depth': 5,
    #           'learning_rate': 0.024093830590526,
    #           'n_estimators': 224,
    #           'min_child_weight': 6,
    #           'gamma':  0.005342041600216,
    #           'subsample': 0.635588411140476,
    #           'colsample_bytree': 0.998857049787802}
    
    xgb_model = xgb.XGBRegressor(** params)

    trained_xgb_model = MultiOutputRegressor(xgb_model).fit(X_train , y_train)

    prediction = trained_xgb_model.predict(X_test)

    mape_test = mean_absolute_percentage_error(prediction, y_test)
    mae_test = mean_absolute_error(prediction, y_test)
    rmse_test = mean_squared_error(prediction, y_test, squared = False)

    print('----')
    print('test MAPE, MAE, RMSE:')

    print(mape_test)
    print(mae_test)
    print(rmse_test)

    print('----')

    mape_t.append(mape_test)
    mae_t.append(mae_test)
    rmse_t.append(rmse_test)



val MAPE, MAE, RMSE:
0.10624548845816087
3.5611293050977917
5.139260333803817
----
test MAPE, MAE, RMSE:
0.08419537646581533
3.028277209726733
4.13499972885405
----
val MAPE, MAE, RMSE:
0.0682377275192539
2.6526203446679406
3.5242013247507447
----
test MAPE, MAE, RMSE:
0.12512136579509944
3.0658148950705475
7.532418206983646
----
val MAPE, MAE, RMSE:
0.12465460257555999
2.127865969731612
2.7871268843485146
----
test MAPE, MAE, RMSE:
0.09328873509342106
2.050405872558391
2.949560705283904
----
val MAPE, MAE, RMSE:
0.07535471470454384
2.2371403544997244
3.1863685429191517
----
test MAPE, MAE, RMSE:
0.1106172285783314
2.307482436602478
3.4348947234257383
----
val MAPE, MAE, RMSE:
0.11017453831237958
1.950156140017199
2.8563214092753504
----
test MAPE, MAE, RMSE:
0.08855821508274514
2.581419974087197
3.76658622549937
----


In [37]:
print(sum(mape)/5)
print(sum(mae)/5)
print(sum(rmse)/5)

0.09693341431397964
2.5057824228028536
3.4986556990195155


In [42]:
print('val')
print(sum(mape)/5)
print(sum(mae)/5)
print(sum(rmse)/5)

print('test')
print(sum(mape_t)/5)
print(sum(mae_t)/5)
print(sum(rmse_t)/5)



val
0.09693341431397964
2.5057824228028536
3.4986556990195155
test
0.10035618420308248
2.6066800776090693
4.363691918009342


# 12 horas - validation + testing

In [43]:
estacion = 4

mape = []
rmse = []
mae = []

mape_t = []
rmse_t = []
mae_t = []

variables = ['MP1', 'MP2_5', 'MP10', 'AQI_MP10', 'AQI_MP2_5']
dependent = ['AQI_MP2_5']

number_of_features = len(variables)

forecast_days = 0.5
training_days = 2
samples_per_day = 288

step = forecast_days*samples_per_day

input_samples = int(samples_per_day * training_days) # cantidad de muestras en 7 dias
output_samples = int(samples_per_day * forecast_days) # cantidad de muestras en 1 dia
train_test_samples = int(input_samples + output_samples) # cantidad de datos para el train_test




train_set_1 = datos[(datos['FECHAHORA']) < (datos.FECHAHORA.min() + relativedelta(months=3))]
# print(train_set_1.info())
# print(train_set_1.FECHAHORA.min())

train_set_2 = datos[datos['FECHAHORA'] < (datos.FECHAHORA.min() + relativedelta(months=6))]
# print(train_set_2.info())
# print(train_set_2.FECHAHORA.min())

train_set_3 = datos[datos['FECHAHORA'] < (datos.FECHAHORA.min() + relativedelta(months=9))]
# print(train_set_3.FECHAHORA.min())

train_set_4 = datos[datos['FECHAHORA'] < (datos.FECHAHORA.min() + relativedelta(months=12))]
#print(train_set_4.FECHAHORA.min())

train_set_5 = datos[datos['FECHAHORA'] < (datos.FECHAHORA.min() + relativedelta(years = 1, months = 3))]
#print(train_set_5.FECHAHORA.min())

test_set_1 = datos[(datos['FECHAHORA'] < (datos.FECHAHORA.min() + relativedelta(months=6)))]

test_set_2 = datos[(datos['FECHAHORA'] < (datos.FECHAHORA.min() + relativedelta(months=9)))]

test_set_3 = datos[(datos['FECHAHORA'] < (datos.FECHAHORA.min() + relativedelta(months=12)))]

test_set_4 = datos[(datos['FECHAHORA'] < (datos.FECHAHORA.min() + relativedelta(months=15)))]

test_set_5 = datos

set_set = [[train_set_1, 2, test_set_1, 3], [train_set_2, 5, test_set_2, 6], [train_set_3, 8, test_set_3, 9], 
           [train_set_4, 11, test_set_4, 12], [train_set_5, 14, test_set_5, 15]]

for i in range(0,5):

    train_months = relativedelta(months = set_set[i][1])
    df_train = set_set[i][0]

    X_train, y_train, X_test, y_test = utils_xgboost.get_everything(df_train, 
                                                        estacion,
                                                        train_months, 
                                                        variables, 
                                                        dependent, 
                                                        train_test_samples, 
                                                        input_samples, 
                                                        output_samples, 
                                                        number_of_features,
                                                        step)
    
    params = {'max_depth': 5,
              'learning_rate': 0.028118418737147,
              'n_estimators': 174,
              'min_child_weight': 6,
              'gamma':  0.00392389768210,
              'subsample': 0.383249592275572,
              'colsample_bytree': 0.760216507183716}
    
    xgb_model = xgb.XGBRegressor(** params)

    trained_xgb_model = MultiOutputRegressor(xgb_model).fit(X_train , y_train)

    # validation

    prediction = trained_xgb_model.predict(X_test)

    mape_val = mean_absolute_percentage_error(prediction, y_test)
    mae_val = mean_absolute_error(prediction, y_test)
    rmse_val = mean_squared_error(prediction, y_test, squared = False)

    print('val MAPE, MAE, RMSE:')

    print(mape_val)
    print(mae_val)
    print(rmse_val)

    mape.append(mape_val)
    mae.append(mae_val)
    rmse.append(rmse_val)

    # testing
    
    train_months = relativedelta(months = set_set[i][3])
    df_train = set_set[i][2]

    X_train, y_train, X_test, y_test = utils_xgboost.get_everything(df_train, 
                                                        estacion,
                                                        train_months, 
                                                        variables, 
                                                        dependent, 
                                                        train_test_samples, 
                                                        input_samples, 
                                                        output_samples, 
                                                        number_of_features,
                                                        step)
    
    # # params = {'max_depth': 5,
    #           'learning_rate': 0.024093830590526,
    #           'n_estimators': 224,
    #           'min_child_weight': 6,
    #           'gamma':  0.005342041600216,
    #           'subsample': 0.635588411140476,
    #           'colsample_bytree': 0.998857049787802}
    
    xgb_model = xgb.XGBRegressor(** params)

    trained_xgb_model = MultiOutputRegressor(xgb_model).fit(X_train , y_train)

    prediction = trained_xgb_model.predict(X_test)

    mape_test = mean_absolute_percentage_error(prediction, y_test)
    mae_test = mean_absolute_error(prediction, y_test)
    rmse_test = mean_squared_error(prediction, y_test, squared = False)

    print('----')
    print('test MAPE, MAE, RMSE:')

    print(mape_test)
    print(mae_test)
    print(rmse_test)

    print('----')

    mape_t.append(mape_test)
    mae_t.append(mae_test)
    rmse_t.append(rmse_test)


val MAPE, MAE, RMSE:
0.1907580475261047
5.96011123428441
8.862502627099563
----
test MAPE, MAE, RMSE:
0.1421206813974915
5.419319690827773
7.022219811727332
----
val MAPE, MAE, RMSE:
0.10928715258178658
4.529505797588464
5.8745703439132875
----
test MAPE, MAE, RMSE:
0.1867957973361338
4.993113176819702
10.233168993040133
----
val MAPE, MAE, RMSE:
0.19070427997210782
3.5303019098138715
4.408990779153277
----
test MAPE, MAE, RMSE:
0.14448701349456702
3.496045252903407
5.071356564982491
----
val MAPE, MAE, RMSE:
0.1314448471870656
4.216131709298195
6.272387004405862
----
test MAPE, MAE, RMSE:
0.17106004127923666
3.5234870070080597
4.900100799176308
----
val MAPE, MAE, RMSE:
0.169111826984098
2.860536090653352
3.826920361164021
----
test MAPE, MAE, RMSE:
0.14397189469824634
4.4419672095481255
6.384450705233636
----


In [44]:
print('val')
print(sum(mape)/5)
print(sum(mae)/5)
print(sum(rmse)/5)

print('test')
print(sum(mape_t)/5)
print(sum(mae_t)/5)
print(sum(rmse_t)/5)


val
0.15826123085023253
4.2193173483276585
5.849074223147202
test
0.15768708564113507
4.374786467421414
6.72225937483198


# 24 horas / validation + testing

In [45]:
estacion = 4

mape = []
rmse = []
mae = []

mape_t = []
rmse_t = []
mae_t = []


# resampling

datos.drop(['ANHO', 'DIA', 'MES', 'HORA', 'MINUTO', 'DIA_TRAF_COD', 'TRAFICO_COD', 'TIPO_COD',
                'MEDICION_DIA', 'MP1_ANTERIOR', 'MP2_5_ANTERIOR', 'MP10_ANTERIOR',
                'TEMPERATURA_PRONOSTICO', 'HUMEDAD_PRONOSTICO', 'PRESION_PRONOSTICO'], axis = 1, inplace = True)

datos = datos.set_index('FECHAHORA', drop = True)

r_i = '30T'
lista_resample = []

for station in range(1,11):
    df_aux = datos[datos['ESTACION'] == station]

    df_aux = df_aux.resample(r_i).mean()
    lista_resample.append(df_aux)

datos = pd.concat(lista_resample)

datos['ANHO'] = datos.index.year
datos['MES'] = datos.index.month
datos['DIA'] = datos.index.day
datos['HORA'] = datos.index.hour
datos['MINUTO'] = datos.index.minute
datos['FECHAHORA'] = datos.index
datos.reset_index(inplace = True, drop = True)


variables = ["ANHO", 'DIA', 'MES', 'HORA', 'MINUTO', 'MP1', 'MP2_5', 'MP10', 'AQI_MP10', 'AQI_MP2_5', 'TIPO', 'TRAFICO', 'HUMEDAD', 
                        'PRESION', 'TEMPERATURA','DIA_SEM']
dependent = ['AQI_MP2_5']

number_of_features = len(variables)

forecast_days = 1
training_days = 2
samples_per_day = 48

step = forecast_days*samples_per_day

input_samples = int(samples_per_day * training_days) # cantidad de muestras en 7 dias
output_samples = int(samples_per_day * forecast_days) # cantidad de muestras en 1 dia
train_test_samples = int(input_samples + output_samples) # cantidad de datos para el train_test


train_set_1 = datos[(datos['FECHAHORA']) < (datos.FECHAHORA.min() + relativedelta(months=3))]
# print(train_set_1.info())
# print(train_set_1.FECHAHORA.min())

train_set_2 = datos[datos['FECHAHORA'] < (datos.FECHAHORA.min() + relativedelta(months=6))]
# print(train_set_2.info())
# print(train_set_2.FECHAHORA.min())

train_set_3 = datos[datos['FECHAHORA'] < (datos.FECHAHORA.min() + relativedelta(months=9))]
# print(train_set_3.FECHAHORA.min())

train_set_4 = datos[datos['FECHAHORA'] < (datos.FECHAHORA.min() + relativedelta(months=12))]
#print(train_set_4.FECHAHORA.min())

train_set_5 = datos[datos['FECHAHORA'] < (datos.FECHAHORA.min() + relativedelta(years = 1, months = 3))]
#print(train_set_5.FECHAHORA.min())

test_set_1 = datos[(datos['FECHAHORA'] < (datos.FECHAHORA.min() + relativedelta(months=6)))]

test_set_2 = datos[(datos['FECHAHORA'] < (datos.FECHAHORA.min() + relativedelta(months=9)))]

test_set_3 = datos[(datos['FECHAHORA'] < (datos.FECHAHORA.min() + relativedelta(months=12)))]

test_set_4 = datos[(datos['FECHAHORA'] < (datos.FECHAHORA.min() + relativedelta(months=15)))]

test_set_5 = datos

set_set = [[train_set_1, 2, test_set_1, 3], [train_set_2, 5, test_set_2, 6], [train_set_3, 8, test_set_3, 9], 
           [train_set_4, 11, test_set_4, 12], [train_set_5, 14, test_set_5, 15]]

for i in range(0,5):

    train_months = relativedelta(months = set_set[i][1])
    df_train = set_set[i][0]

    X_train, y_train, X_test, y_test = utils_xgboost.get_everything(df_train, 
                                                        estacion,
                                                        train_months, 
                                                        variables, 
                                                        dependent, 
                                                        train_test_samples, 
                                                        input_samples, 
                                                        output_samples, 
                                                        number_of_features,
                                                        step)
    
    params = {'max_depth': 5,
              'learning_rate': 0.028118418737147,
              'n_estimators': 174,
              'min_child_weight': 6,
              'gamma':  0.00392389768210,
              'subsample': 0.383249592275572,
              'colsample_bytree': 0.760216507183716}
    
    xgb_model = xgb.XGBRegressor(** params)

    trained_xgb_model = MultiOutputRegressor(xgb_model).fit(X_train , y_train)

    # validation

    prediction = trained_xgb_model.predict(X_test)

    mape_val = mean_absolute_percentage_error(prediction, y_test)
    mae_val = mean_absolute_error(prediction, y_test)
    rmse_val = mean_squared_error(prediction, y_test, squared = False)

    print('val MAPE, MAE, RMSE:')

    print(mape_val)
    print(mae_val)
    print(rmse_val)

    mape.append(mape_val)
    mae.append(mae_val)
    rmse.append(rmse_val)

    # testing
    
    train_months = relativedelta(months = set_set[i][3])
    df_train = set_set[i][2]

    X_train, y_train, X_test, y_test = utils_xgboost.get_everything(df_train, 
                                                        estacion,
                                                        train_months, 
                                                        variables, 
                                                        dependent, 
                                                        train_test_samples, 
                                                        input_samples, 
                                                        output_samples, 
                                                        number_of_features,
                                                        step)
    
    # # params = {'max_depth': 5,
    #           'learning_rate': 0.024093830590526,
    #           'n_estimators': 224,
    #           'min_child_weight': 6,
    #           'gamma':  0.005342041600216,
    #           'subsample': 0.635588411140476,
    #           'colsample_bytree': 0.998857049787802}
    
    xgb_model = xgb.XGBRegressor(** params)

    trained_xgb_model = MultiOutputRegressor(xgb_model).fit(X_train , y_train)

    prediction = trained_xgb_model.predict(X_test)

    mape_test = mean_absolute_percentage_error(prediction, y_test)
    mae_test = mean_absolute_error(prediction, y_test)
    rmse_test = mean_squared_error(prediction, y_test, squared = False)

    print('----')
    print('test MAPE, MAE, RMSE:')

    print(mape_test)
    print(mae_test)
    print(rmse_test)

    print('----')

    mape_t.append(mape_test)
    mae_t.append(mae_test)
    rmse_t.append(rmse_test)


val MAPE, MAE, RMSE:
0.3140261346199862
10.135150297188465
13.480858792694411
----
test MAPE, MAE, RMSE:
0.2385204276676803
9.098171531037892
11.59673231950279
----
val MAPE, MAE, RMSE:
0.16575850830129643
6.278678655133817
7.581406095315518
----
test MAPE, MAE, RMSE:
0.26466867739238603
7.549707416068301
13.743613679004902
----
val MAPE, MAE, RMSE:
0.22441637711488446
4.318481242136349
5.231903136722768
----
test MAPE, MAE, RMSE:
0.2328102028719783
6.171935854460855
7.649998696501136
----
val MAPE, MAE, RMSE:
0.18218692155716162
5.6047066528172715
8.256284060961145
----
test MAPE, MAE, RMSE:
0.29701221612473605
6.98545778403529
9.027430034087201
----
val MAPE, MAE, RMSE:
0.3412092450150182
6.684620620291909
8.083185552467983
----
test MAPE, MAE, RMSE:
0.2684986239616797
8.039213509802465
10.740971985377206
----


In [46]:
print('val')
print(sum(mape)/5)
print(sum(mae)/5)
print(sum(rmse)/5)

print('test')
print(sum(mape_t)/5)
print(sum(mae_t)/5)
print(sum(rmse_t)/5)

val
0.24551943732166942
6.604327493513563
8.526727527632364
test
0.2603020296036921
7.568897219080961
10.551749342894649


In [48]:
print(np.std(mape_t))
print(np.std(mae_t))
print(np.std(rmse_t))

0.02308301664139756
0.9851038738809955
2.100452272202814


In [49]:
mape_6h_t = [0.08419537646581533, 0.12512136579509944, 0.09328873509342106, 0.1106172285783314, 0.08855821508274514]

mae_6h_t = [3.028277209726733, 3.0658148950705475, 2.050405872558391, 2.307482436602478, 2.581419974087197]

rmse_6h_t = [4.13499972885405, 7.532418206983646, 2.949560705283904, 3.4348947234257383,3.76658622549937 ]

print(np.std(mape_6h_t))
print(np.std(mae_6h_t))
print(np.std(rmse_6h_t))

0.015289969718474948
0.39702574597469936
1.6316919400006857


In [51]:
mape_12h_t = [0.1421206813974915, 0.1867957973361338, 0.14448701349456702, 0.17106004127923666, 0.14397189469824634]

mae_12h_t = [5.419319690827773,4.993113176819702, 3.496045252903407, 3.5234870070080597, 4.4419672095481255]

rmse_12h_t = [7.022219811727332,10.233168993040133, 5.071356564982491, 4.900100799176308, 6.384450705233636]

print(np.std(mape_12h_t))
print(np.std(mae_12h_t))
print(np.std(rmse_12h_t))

0.01805997977513935
0.7713349103807424
1.9275019725328022
